In [221]:
import numpy as np
import emo_utils
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [222]:
X_train, Y_train = emo_utils.read_csv('/Users/Moli/Documents/MaterialsML/deeplearning.ai-master/02-课后作业/05-第五课 序列模型/02-第五课第二周作业/Emojify/data/train_emoji.csv')
X_test, Y_test = emo_utils.read_csv('/Users/Moli/Documents/MaterialsML/deeplearning.ai-master/02-课后作业/05-第五课 序列模型/02-第五课第二周作业/Emojify/data/test_emoji.csv')

maxLen = len(max(X_train, key=len).split())

In [223]:
#X_train

In [224]:
index  = 3
print(X_train[index], emo_utils.label_to_emoji(Y_train[index]))

Miss you so much ❤️


In [225]:
Y_oh_train = emo_utils.convert_to_one_hot(Y_train, C=5)
Y_oh_test = emo_utils.convert_to_one_hot(Y_test, C=5)

In [226]:
#Y_oh_train

In [227]:
#np.eye(3)[2]

In [228]:
word_to_index, index_to_word, word_to_vec_map = emo_utils.read_glove_vecs('/Users/Moli/Documents/MaterialsML/deeplearning.ai-master/02-课后作业/05-第五课 序列模型/02-第五课第二周作业/Emojify/data/glove.6B.50d.txt')

In [229]:
word = "cucumber"
index = 113317
print("单词{0}对应的索引是：{1}".format(word, word_to_index[word]))
print("索引{0}对应的单词是：{1}".format(index, index_to_word[index]))

单词cucumber对应的索引是：113317
索引113317对应的单词是：cucumber


In [230]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    将句子转换为单词列表，提取其GloVe向量，然后将其平均。
    
    参数：
        sentence -- 字符串类型，从X中获取的样本。
        word_to_vec_map -- 字典类型，单词映射到50维的向量的字典
        
    返回：
        avg -- 对句子的均值编码，维度为(50,)
    """
    
    # 第一步：分割句子，转换为列表。
    words = sentence.lower().split()
    
    # 初始化均值词向量
    avg = np.zeros(50,)
    
    # 第二步：对词向量取平均。
    for w in words:
        avg += word_to_vec_map[w]
    avg = np.divide(avg, len(words))
    
    return avg

In [231]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = ", avg)

avg =  [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


In [232]:
def model(X, Y, word_to_vec_map, learning_rate=0.01, num_iterations=400):
    """
    在numpy中训练词向量模型。
    
    参数：
        X -- 输入的字符串类型的数据，维度为(m, 1)。
        Y -- 对应的标签，0-7的数组，维度为(m, 1)。
        word_to_vec_map -- 字典类型的单词到50维词向量的映射。
        learning_rate -- 学习率.
        num_iterations -- 迭代次数。
        
    返回：
        pred -- 预测的向量，维度为(m, 1)。
        W -- 权重参数，维度为(n_y, n_h)。
        b -- 偏置参数，维度为(n_y,)
    """
    np.random.seed(1)
    
    # 定义训练数量
    m = Y.shape[0]
    n_y = 5
    n_h = 50
    
    # 使用Xavier初始化参数
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # 将Y转换成独热编码
    Y_oh = emo_utils.convert_to_one_hot(Y, C=n_y)
    
    # 优化循环
    for t in range(num_iterations):
        for i in range(m):
            # 获取第i个训练样本的均值
            avg = sentence_to_avg(X[i], word_to_vec_map)
            
            # 前向传播
            z = np.dot(W, avg) + b
            a = emo_utils.softmax(z)
            
            # 计算第i个训练的损失
            cost = -np.sum(Y_oh[i]*np.log(a))
            
            # 计算梯度
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz
            
            # 更新参数
            W = W - learning_rate * dW
            b = b - learning_rate * db
        if t % 100 == 0:
            print("第{t}轮，损失为{cost}".format(t=t,cost=cost))
            pred = emo_utils.predict(X, Y, W, b, word_to_vec_map)
            
    return pred, W, b


In [233]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)


第0轮，损失为1.9520498812810072
Accuracy: 0.3484848484848485
第100轮，损失为0.07971818726014807
Accuracy: 0.9318181818181818
第200轮，损失为0.04456369243681402
Accuracy: 0.9545454545454546
第300轮，损失为0.03432267378786059
Accuracy: 0.9696969696969697


In [234]:
print("=====训练集====")
pred_train = emo_utils.predict(X_train, Y_train, W, b, word_to_vec_map)
print("=====测试集====")
pred_test = emo_utils.predict(X_test, Y_test, W, b, word_to_vec_map)


=====训练集====
Accuracy: 0.9772727272727273
=====测试集====
Accuracy: 0.625


In [235]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [236]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    输入的是X（字符串类型的句子的数组），再转化为对应的句子列表，
    输出的是能够让Embedding()函数接受的列表或矩阵（参见图4）。
    
    参数：
        X -- 句子数组，维度为(m, 1)
        word_to_index -- 字典类型的单词到索引的映射
        max_len -- 最大句子的长度，数据集中所有的句子的长度都不会超过它。
        
    返回：
        X_indices -- 对应于X中的单词索引数组，维度为(m, max_len)
    """
    
    m = X.shape[0]  # 训练集数量
    # 使用0初始化X_indices
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
        # 将第i个居住转化为小写并按单词分开。
        sentences_words = X[i].lower().split()
        
        # 初始化j为0
        j = 0
        
        # 遍历这个单词列表
        for w in sentences_words:
            # 将X_indices的第(i, j)号元素为对应的单词索引
            X_indices[i, j] = word_to_index[w]
            
            j += 1
            
    return X_indices

In [237]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [238]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    创建Keras Embedding()层，加载已经训练好了的50维GloVe向量
    
    参数：
        word_to_vec_map -- 字典类型的单词与词嵌入的映射
        word_to_index -- 字典类型的单词到词汇表（400,001个单词）的索引的映射。
        
    返回：
        embedding_layer() -- 训练好了的Keras的实体层。
    """
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    # 初始化嵌入矩阵
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # 将嵌入矩阵的每行的“index”设置为词汇“index”的词向量表示
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    # 定义Keras的embbeding层
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    # 构建embedding层。
    embedding_layer.build((None,))
    
    # 将嵌入层的权重设置为嵌入矩阵。
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [239]:
word_to_index['cucumber']

113317

In [240]:
len(word_to_index)

400000

In [241]:
type(word_to_index)

dict

In [242]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])
#word_to_index

weights[0][1][3] = -0.3403


In [243]:
#embedding_layer.get_weights()[1]

In [244]:
word_to_vec_map["cucumber"].shape[0]

50

In [245]:
embedding_layer

In [246]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.

    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')

    # Create the embedding layer pretrained with GloVe Vectors (鈮? line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=True)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(5, activation='softmax')(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices ,outputs=X)

    ### END CODE HERE ###

    return model


In [247]:
model = Emojify_V2((10,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 10)                0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 10, 50)            20000050  
_________________________________________________________________
lstm_22 (LSTM)               (None, 10, 128)           91648     
_________________________________________________________________
dropout_21 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 10, 128)           131584    
_________________________________________________________________
dropout_22 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 128)               131584    
__________

In [248]:
maxLen = len(max(X_train, key=len).split())

In [249]:
maxLen

10

In [250]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [251]:
X_train_indices = sentences_to_indices(X_train, word_to_index, 10)
Y_train_oh = emo_utils.convert_to_one_hot(Y_train, C = 5)


In [252]:
model.fit(X_train_indices, Y_train_oh, epochs = 100, batch_size = 32, shuffle=True)


Epoch 1/100
132/132 [==============================] - 5s 40ms/step - loss: 1.6048 - acc: 0.2121
Epoch 2/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5868 - acc: 0.3030
Epoch 3/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5731 - acc: 0.2652
Epoch 4/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5542 - acc: 0.3485
Epoch 5/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5457 - acc: 0.3106
Epoch 6/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5085 - acc: 0.3636
Epoch 7/100
132/132 [==============================] - 0s 2ms/step - loss: 1.5095 - acc: 0.3712
Epoch 8/100
132/132 [==============================] - 0s 2ms/step - loss: 1.4713 - acc: 0.5455
Epoch 9/100
132/132 [==============================] - 0s 2ms/step - loss: 1.4090 - acc: 0.5227
Epoch 10/100
132/132 [==============================] - 0s 2ms/step - loss: 1.3603 - acc: 0.5530
Epoch 11/100
132/132 [================

132/132 [==============================] - 0s 2ms/step - loss: 0.9692 - acc: 0.9318
Epoch 86/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9430 - acc: 0.9545
Epoch 87/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9689 - acc: 0.9318
Epoch 88/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9617 - acc: 0.9394
Epoch 89/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9557 - acc: 0.9470
Epoch 90/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9539 - acc: 0.9545
Epoch 91/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9525 - acc: 0.9545
Epoch 92/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9482 - acc: 0.9621
Epoch 93/100
132/132 [==============================] - 0s 2ms/step - loss: 0.9550 - acc: 0.9470
Epoch 94/100
132/132 [==============================] - 0s 3ms/step - loss: 0.9749 - acc: 0.9318
Epoch 95/100
132/132 [=====================

In [253]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = emo_utils.convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)

print("Test accuracy = ", acc)


56/56 [==============================] - 1s 25ms/step
Test accuracy =  0.5714285714285714
